In [1]:
import numpy as np
from unray.datawidgets import *
from unray.plotwidgets import *
from unray.newwidgets import *
from ipydatawidgets import *
import pythreejs as three

In [2]:
filename = "data/heart.npz"
#filename = "data/brain.npz"
#filename = "data/aneurysm.npz"

mesh_data = np.load(filename)
cells_array = mesh_data["cells"].astype(np.int32)
points_array = mesh_data["points"].astype(np.float32)

# Coordinates of all vertices in mesh
x = list(points_array.T)  # x[2] = z coordinate array for all vertices

# Model center 3d vector
center = list(map(lambda x: x.mean(), x))

# Model min/max coordinates
bbox = list(map(lambda x: (x.min(), x.max()), x))

# Coordinates with origo shifted to center of model
xm = list(map(lambda x, mp: x - mp, x, center))

# Distance from model center
xd = np.sqrt(sum(map(lambda x: x**2, xm)))
radius = xd.max()

# Distance from center, normalized to max 1.0
func_dist = xd / radius

# A constant for all vertices
func_const = np.ones(x[0].shape)

# x coordinate
func_x = x[0]

# A wave pattern from the center of the model
freq = 4
func_wave = 2.0 + np.sin((freq * 2 * np.pi / radius) * xd)

In [3]:
cells = NDArrayWidget(cells_array)
points = NDArrayWidget(points_array)
mesh = Mesh(cells=cells, points=points)
field = Field(mesh=mesh, values=func_wave)
lut = ArrayColorLUT(values=[[1,0,0], [0,0,1]])

In [4]:
width = 800
height = 600
camera = three.PerspectiveCamera(position=[10, 10, 10], aspect=width/height)
key_light = three.DirectionalLight(position=[0, 10, 10])
ambient = three.AmbientLight(intensity=0.5)
scene = three.Scene(children=[key_light, ambient, camera], background='gray')
controls = three.OrbitControls(camera)
renderer = three.Renderer(scene, camera, [controls],
                          width=width, height=height)
renderer

A Jupyter Widget

In [5]:
plot = surf(field, lut=lut)
scene.add(plot)
plot.position = [0, 9, 0]

In [6]:
box = three.Mesh(three.BoxGeometry(1, 1, 1), three.MeshLambertMaterial(), position=[5, 0, 0])
scene.add(box)

In [7]:
renderer.background = 'white'

In [9]:
C = ColorConstant()
plot2 = XrayPlot(density=field, color=C)
scene.add(plot2)

In [10]:
plot0 = scene.children[3]
plot1 = scene.children[4]
plot2 = scene.children[6]

In [11]:
plot0.visible = False
plot1.visible = False

In [32]:
[(i, type(c)) for i, c in enumerate(scene.children)]

[(0, pythreejs.lights.DirectionalLight_autogen.DirectionalLight),
 (1, pythreejs.lights.AmbientLight_autogen.AmbientLight),
 (2, pythreejs.cameras.PerspectiveCamera_autogen.PerspectiveCamera),
 (3, unray.newwidgets.SurfacePlot),
 (4, unray.newwidgets.SurfacePlot),
 (5, pythreejs.objects.Mesh_autogen.Mesh),
 (6, unray.newwidgets.XrayPlot)]